Seal classification

In [ ]:
#pip install requests
!pip install tensorflow h5py -q


In [ ]:
import sys
print(sys.version)

3.10.12 (main, Jun 11 2023, 05:26:28) [GCC 11.4.0]


In [1]:
import cv2
import numpy as np

from keras.models import load_model
from keras.models import model_from_json
from sklearn.preprocessing import MultiLabelBinarizer
from PIL import Image


In [6]:
def seal_verification(image_parth):


  # Load the saved model
  with open('/content/drive/MyDrive/Work space/Silverline IT/Project/Healing_minds!/API/app/Services02/fun2_models/seal classification model 1/model_architecture.json', 'r') as json_file:
    model_json = json_file.read()

  model =model_from_json(model_json)
  model.load_weights('/content/drive/MyDrive/Work space/Silverline IT/Project/Healing_minds!/API/app/Services02/fun2_models/seal classification model 1/Seal_classification2_best.h5')

  #local image load
  #img_path =f'/content/drive/MyDrive/Work space/Data set/Healing Minds/fun2/predict data/IMG_20231020_235352.jpg'
  img_path =image_parth



  # Load the test image
  image = cv2.imread(img_path)

  # Preprocess the image
  if image is not None:
      image = cv2.resize(image, (200, 200))
      image = image.astype('float32') / 255.0
      image= cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
      image = np.expand_dims(image, axis=0)


  else:
      print("Invalid or empty image.")



  # Make predictions
  predictions = model.predict(image)
  #print(predictions)

  # Decode the predictions
  # Check the shape of predictions
  #print("Shape of predictions:", predictions[0].shape)


  mlb = MultiLabelBinarizer()
  mlb.classes_=['No seal', 'seal']

  seal_classe=[]
  noseal_classe=[]
  score=[]
  # Ensure mlb.classes_ matches the number of classes predicted
  if len(mlb.classes_) == predictions[0].shape[0]:
      for i in range(len(predictions[0])):
          if predictions[0][i] >= 0.75:
              print(f"{mlb.classes_[i]}: {round(predictions[0][i] * 100, 2)}%")

              if mlb.classes_[i]=="seal":
                seal_classe.append(mlb.classes_[i])
                score.append(predictions[0][i]*100)

              else:

                noseal_classe.append(mlb.classes_[i])
                score.append(predictions[0][i]*100)

          else:
            pass

  else:
      print("Number of classes in mlb.classes_ doesn't match the model's output.")

  dictionary=dict()

  if len(seal_classe) !=0:
    dictionary= {"text": "Valid seal verification is included in this document","seal Class": seal_classe, "validation score":score}

  else:
    dictionary= {"text": "Valid seal verification is not included in this document","seal Class":noseal_classe, "validation score":score}

  return dictionary



In [7]:
test=seal_verification("/content/drive/MyDrive/Work space/Data set/Healing Minds/fun2/predict data/IMG_20231020_235352.jpg")

1/1 [==============================] - 1s 670ms/step
No seal: 98.71%


In [8]:
print(test)

{'text': 'Valid seal verification is not included in this document', 'seal Class': ['No seal'], 'validation score': [98.70935082435608]}


image segmentation

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

from skimage.filters import threshold_local
from PIL import Image

In [ ]:
def image_segmentation(image_path):


  def opencv_resize(image, ratio):
      width = int(image.shape[1] * ratio)
      height = int(image.shape[0] * ratio)
      dim = (width, height)
      return cv2.resize(image, dim, interpolation = cv2.INTER_AREA)

  def plot_rgb(image):
      plt.figure(figsize=(16,10))
      return plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))


  def plot_gray(image):
      plt.figure(figsize=(16,10))
      return plt.imshow(image, cmap='Greys_r')

  # Sample file out of the dataset
  file_name =image_path #input image
  img = Image.open(file_name)
  img.thumbnail((800,800), Image.ANTIALIAS)

  image = cv2.imread(file_name)
  # Downscale image as finding receipt contour is more efficient on a small image
  resize_ratio = 500 / image.shape[0]
  original = image.copy()
  image = opencv_resize(image, resize_ratio)

  # Convert to grayscale for further processing
  gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

  blurred = cv2.GaussianBlur(gray, (5, 5), 0)

  rectKernel = cv2.getStructuringElement(cv2.MORPH_RECT, (9, 9))
  dilated = cv2.dilate(blurred, rectKernel)

  edged = cv2.Canny(dilated, 100, 200, apertureSize=3)

  contours, hierarchy = cv2.findContours(edged, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
  image_with_contours = cv2.drawContours(image.copy(), contours, -1, (0,255,0), 3)


  largest_contours = sorted(contours, key = cv2.contourArea, reverse = True)[:10]
  image_with_largest_contours = cv2.drawContours(image.copy(), largest_contours, -1, (0,255,0), 3)



  # approximate the contour by a more primitive polygon shape
  def approximate_contour(contour):
      peri = cv2.arcLength(contour, True)
      return cv2.approxPolyDP(contour, 0.032 * peri, True)

  def get_receipt_contour(contours):
    # loop over the contours
    for c in contours:
        approx = approximate_contour(c)
        # if our approximated contour has four points, we can assume it is receipt's rectangle
        if len(approx) == 4:
            return approx

  get_receipt_contour(largest_contours)
  receipt_contour = get_receipt_contour(largest_contours)
  image_with_receipt_contour = cv2.drawContours(image.copy(), [receipt_contour], -1, (0, 255, 0), 2)

  def contour_to_rect(contour):
      pts = contour.reshape(4, 2)
      rect = np.zeros((4, 2), dtype = "float32")
      # top-left point has the smallest sum
      # bottom-right has the largest sum
      s = pts.sum(axis = 1)
      rect[0] = pts[np.argmin(s)]
      rect[2] = pts[np.argmax(s)]
      # compute the difference between the points:
      # the top-right will have the minumum difference
      # the bottom-left will have the maximum difference
      diff = np.diff(pts, axis = 1)
      rect[1] = pts[np.argmin(diff)]
      rect[3] = pts[np.argmax(diff)]
      return rect / resize_ratio



  def wrap_perspective(img, rect):
      # unpack rectangle points: top left, top right, bottom right, bottom left
      (tl, tr, br, bl) = rect
      # compute the width of the new image
      widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
      widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
      # compute the height of the new image
      heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
      heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
      # take the maximum of the width and height values to reach
      # our final dimensions
      maxWidth = max(int(widthA), int(widthB))
      maxHeight = max(int(heightA), int(heightB))
      # destination points which will be used to map the screen to a "scanned" view
      dst = np.array([
          [0, 0],
          [maxWidth - 1, 0],
          [maxWidth - 1, maxHeight - 1],
          [0, maxHeight - 1]], dtype = "float32")
      # calculate the perspective transform matrix
      M = cv2.getPerspectiveTransform(rect, dst)
      # warp the perspective to grab the screen
      return cv2.warpPerspective(img, M, (maxWidth, maxHeight))


  scanned = wrap_perspective(original.copy(), contour_to_rect(receipt_contour))

  def bw_scanner(image):
      gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
      T = threshold_local(gray, 21, offset = 5, method = "gaussian")
      return (gray > T).astype("uint8") * 255
  result = bw_scanner(scanned)


  output = Image.fromarray(result)
  out_image=output.save('result.png')
  return out_image



In [ ]:
test=image_segmentation('/content/drive/MyDrive/Work space/Data set/Healing Minds/fun2/predict data/IMG_20231020_235345_HDR.jpg')

<ipython-input-74-36ef21d3384f>:22: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  img.thumbnail((800,800), Image.ANTIALIAS)


OCR

In [ ]:
#install easyocr
!pip install easyocr -q
import easyocr
import numpy as np
import cv2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 21.5 MB/s eta 0:00:00


In [ ]:
!pip show easyocr

Name: easyocr
Version: 1.7.1
Summary: End-to-End Multi-Lingual Optical Character Recognition (OCR) Solution
Home-page: https://github.com/jaidedai/easyocr
Author: Rakpong Kittinaradorn
Author-email: r.kittinaradorn@gmail.com
License: Apache License 2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: ninja, numpy, opencv-python-headless, Pillow, pyclipper, python-bidi, PyYAML, scikit-image, scipy, Shapely, torch, torchvision
Required-by: 


In [ ]:
def easy_ocr(easy_image):

  # Initialize the reader with the desired language(s)
  ocr = easyocr.Reader(["en"])

  img_path =easy_image
  # Load the test image
  image = cv2.imread(img_path)

  if image is not None:

    # Perform OCR on the image
    #add file path or url
    results =ocr.readtext(img_path)

    #get the string to the list
    all_content=np.array([])
    for element in results:
        all_content= np.append(all_content,element[1])


    #combine the result
    text = " ".join(all_content)
    text=text.capitalize()

  else:
    pass

  return text


In [ ]:
test2=easy_ocr("/content/drive/MyDrive/Work space/Data set/Healing Minds/fun2/predict data/IMG_20231020_235352.jpg")

In [ ]:
print(test2)

Wc ceylinco general insurance vp 'ceylinco house' 69, janadhipathi ltdeek outhe jert mawatta; coombo 1 certificate of insurance vehicie no wpgj8644 mcclh229046371 make & vodel yavaha dt125 policy %o pp00213f0008569 name m.k.g. rupasinghe address no : 233/9/1 jaya mw kudamaduwa siddamulla period of cover 16-mar-2023 to 16-mar-2024 engine no 3fw024477 chassis no 2fw024477 16mar subjedtto terrns and conditions specifed in tne policy docurrient


Keras ocr

In [ ]:
!pip install keras-ocr -q



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 55.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 kB 63.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 6.3 MB/s eta 0:00:00


In [ ]:
import keras_ocr
import matplotlib.pyplot as plt
import pandas as pd
from keras_ocr import pipeline

In [ ]:
!pip show opencv-python

Name: opencv-python
Version: 4.8.0.76
Summary: Wrapper package for OpenCV python bindings.
Home-page: https://github.com/opencv/opencv-python
Author: 
Author-email: 
License: Apache 2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: numpy, numpy, numpy, numpy
Required-by: dopamine-rl, imgaug


In [ ]:
 #create the pipeline
 pipeline=pipeline.Pipeline()

Looking for /root/.keras-ocr/craft_mlt_25k.h5
Looking for /root/.keras-ocr/crnn_kurapan.h5


In [ ]:
def keras_ocr(keras_image,pipeline=pipeline):

  # Replace this with your actual image path
  img_path =keras_image

  # Load the test image
  image = cv2.imread(img_path)


  if image is not None:

    #Get Predictions use recognize
    results = pipeline.recognize([image])

    #get the prediction to the data frame
    text_df=pd.DataFrame(results[0], columns=['text', 'bbox'])

    #Re arrange the data set to list
    L=[]
    for i in text_df['text']:
      L.append(i)

    #combile the list
    text=" ".join(L)

    text=text.capitalize()

  else:
    pass

  return text

In [ ]:
test6=keras_ocr("/content/drive/MyDrive/Work space/Data set/Healing Minds/fun2/predict data/IMG_20231020_235352.jpg")

2/2 [==============================] - 18s 7s/step


In [ ]:
print(test6)

Cesinc ceylinco general insurance lidnud vid ceyinco house 69 janadhipath manatha coombol onele srat certificate of insurance vehicle no wpgje644 mcclh2z6616711 make r wodel yamaha dt125 policy no ppoo21f00666 name mre kg rupasinghe address no 233191 jaya mw kudamaduwa siddamulla igmarzo23 period to 5marz024 cover of 3fwo24477 engine no ismar 2fw024477 chassis no docurnent the policy spected in conditons subjec and to terms


Scoring

In [ ]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
nltk.download('punkt')
from dateutil.parser import parse
from datetime import datetime
import spacy
from spacy.matcher import PhraseMatcher
from spacy.matcher import Matcher


# load spacy model
#nlp = spacy.load('en_core_web_lg')
#nlp = spacy.load('en_core_web_md')
nlp = spacy.load('en_core_web_sm')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
def score_fun(full_name:str,name_inti:str,address:str,birth_day:str,text:str):

    #create the basics ,before spacy model
    unwanted_symbols=[".","-",",","_",":-",":",";","%"]
    unwanted_words=["no-","no:","no",".",",","-","_",":-",":",";","%","mawatha","mawata","mavatha","mawata","mv","mw","sri lanka","sri",
                    "lanka","para","road","rd","no:-","no;-","no;",'handiya',"handhiya"]

    #split full name
    full_name=full_name.lower()
    split_name=full_name.split(" ")


    #remove unwanted symbols
    name_inti=name_inti.lower()
    cleaned_name=""
    for char in name_inti:
        if char not in unwanted_symbols:
            cleaned_name += char


    #address
    address=address.lower()

    #token address
    token_addres = nltk.word_tokenize(address)
    #print (token_addres)

    #remove unwanted words
    address_split=[]
    for word in token_addres:
      if word not in unwanted_words:
        address_split.append(word)


    #date format changing
    birth_day=birth_day.lower()
    YMD=[]
    #from dateutil.parser import parse
    def convert_to_YMD(date_string):
        try:
            # Parse the date string using dateutil
            parsed_date = parse(date_string)

            # Convert it to Y-M-D format
            YMD_date = parsed_date.strftime("%Y-%m-%d")

            return YMD_date
        except ValueError:
            return None

    # Test with different date formats
    date_strings = [birth_day]

    for date_string in date_strings:
        converted_date = convert_to_YMD(date_string)
        if converted_date:
            YMD.append(converted_date)
        else:
            print(f"Date: {date_string} has an incorrect format.")




    full_name=full_name
    split_name=split_name
    name_inti=name_inti
    cleaned_name=cleaned_name
    address=address
    address_split=address_split
    YMD=YMD

    #return variable
    name_final=[]
    address_final=[]
    age_final=[]

######################full string matched function#####################
    #match the full string of the variation name and address
    def full_match(text,variation):
      score=0
      l=[]

      #create matched items to need
      matcher = PhraseMatcher(nlp.vocab)
      #create matched items to need
      lang_list=[variation]


      #create the pattern list
      # Only run nlp.make_doc to speed things up
      patterns = [nlp(lang) for lang in lang_list]

      #add pattern to the matche
      matcher.add("TerminologyList", patterns)

      doc=nlp(text)
      matches=matcher(doc)
      #matched text append to l

      for matcher_id,start,end in matches:
        span=doc[start:end]
        #print(span.text)
        l.append(span.text)

      #remove repeate eliment
      l=list(set(l))
      #print(l)

      #matched 100% ok with variation
      for i in l:
        for j in lang_list:
          if i==j:
            #print(100)
            score=100
          else:
            print("no 100% mathced with variable")



      if score== 100:
        return score

      else:
        return 0

################## split string mached function ###################
    def split_match(text,variation):
      l=[]
      #create matched items to need
      matcher = PhraseMatcher(nlp.vocab)
      lang_list=variation

      # Only run nlp.make_doc to speed things up
      patterns = [nlp(lang) for lang in lang_list]
      matcher.add("TerminologyList", patterns)

      doc = nlp(text)
      matches = matcher(doc)

      for match_id, start, end in matches:
        span = doc[start:end]
        #print(span.text)
        l.append(span.text)

      #print(l)


      l=list(set(l))
      #print(l)

      matched_list=[]
      #percentage_score=0
      for i in l:
        for j in lang_list:
          if i==j:
            matched_list.append(i)


      #print(matched_list)


      percentage_score=len(matched_list)/len(lang_list)*100
      #print(percentage_score)
      return percentage_score

############################ AGE ################
    def age(YMD,text):
      #final birthday score list
      birth_day_score=[]

      #get the date in text
      # load spacy model
      #nlp = spacy.load('en_core_web_lg')
      #nlp = spacy.load('en_core_web_md')
      nlp = spacy.load('en_core_web_sm')

      # load data
      sentence = text
      doc = nlp(sentence)
      date=[]

      # print entities
      for ent in doc.ents:
        if ent.label_=="DATE":
          #print(ent.label_,ent.text)
          date.append(ent.text)

        elif ent.label_=="CARDINAL":
          #print(ent.label_,ent.text)
          date.append(ent.text)

      #print(date)
      #Match the text date and YMD
      text_YMD=[]


      #from dateutil.parser import parse
      def convert_to_YMD(date_string):
        try:
          # Parse the date string using dateutil
          parsed_date = parse(date_string)

          # Convert it to Y-M-D format
          YMD_date = parsed_date.strftime("%Y-%m-%d")

          return YMD_date
        except ValueError:
          return None

      # Test with different date formats
      date_strings = date

      for date_string in date_strings:
        converted_date = convert_to_YMD(date_string)
        if converted_date:
            text_YMD.append(converted_date)
        else:
            print(f"Date: {date_string} has an incorrect format.")

      #print(text_YMD)

      for dt in text_YMD:
        for sd in YMD:
          if dt == sd:
            birth_day_score.append(100)

          else:
            birth_day_score.append(0)



      ###check tha birth day in text
      l=[]
      #create matched items to need
      matcher = PhraseMatcher(nlp.vocab)
      lang_list=YMD

      # Only run nlp.make_doc to speed things up
      patterns = [nlp(lang) for lang in lang_list]
      matcher.add("TerminologyList", patterns)

      doc = nlp(text)
      matches = matcher(doc)

      for match_id, start, end in matches:
        span = doc[start:end]
        #print(span.text)
        l.append(span.text)

      #print(l)


      l=list(set(l))
      #print(l)

      for i in l:
        for j in lang_list:
          if i==j:
            birth_day_score.append(100)
          else:
            birth_day_score.append(0)



      ###search age in text
      #from datetime import datetime
      #from spacy.matcher import Matcher

      # Assuming the date of birth in the format "YYYY-MM-DD"
      date_of_birth = " "
      for b in YMD:
        date_of_birth=b

      # Parse the date of birth string into a datetime object
      dob = datetime.strptime(date_of_birth, "%Y-%m-%d")

      # Get the current date
      current_date = datetime.now()

      # Calculate the age
      age = current_date.year - dob.year - ((current_date.month, current_date.day) < (dob.month, dob.day))
      #print(age)

      ###check the age in text

      matcher = Matcher(nlp.vocab)
      pattern = [{"LOWER": "age"}, {"IS_PUNCT": True}, {"IS_DIGIT": True}]
      matcher.add("Test", [pattern])
      doc = nlp(text)
      matches = matcher(doc)


      age_text=""
      for matcher_id,start,end in matches:
        #print(doc[start:end])
        age_text=doc[start:end]
        #print(age_text)

      final_age=""
      tokens_age = nltk.word_tokenize(str(age_text))
      for q in tokens_age:
        if q==str(age):
          birth_day_score.append(100)
        else:
          birth_day_score.append(0)



      ###Birthday final score
      final_B_score=0
      for s in birth_day_score:
        if s==100:
          final_B_score=s
          break

        else:
          final_B_score=0



      #print(birth_day_score)
      #return age final score
      return final_B_score





#@@@@@@@@@@@@@@@@@@@@@@ method compile and get final scores @@@@@@@@@@@@@@@@@@@@@@#

    full_name_score=full_match(text,full_name)
    name_inti_score=full_match(text,name_inti)
    cleaned_name_score=full_match(text,cleaned_name)
    split_name_score=split_match(text,split_name)


    address_score=full_match(text,address)
    address_split_score=split_match(text,address_split)

    age_score=age(YMD,text)


    #name return final score
    name_final.append(full_name_score)
    name_final.append(name_inti_score)
    name_final.append(cleaned_name_score)
    name_final.append(split_name_score)

    #name address final score
    address_final.append(address_score)
    address_final.append(address_split_score)

    #age address final score
    age_final.append(age_score)




    return  max(name_final),max(address_final),max( age_final)











In [ ]:
test5=score_fun("kaushika gihan Rupasinghe","K.G Rupasinghe"," No- 233/9/1, Jayamawatha rd,kudamaduwa, siddamulla,piliyandala,sri lanka ","1995/16/oct","cetlinc ceylinco general age 28 insurance lidnud vip ceylinco house 69 janadhipath manatha coombol ount sran certificate of insurance vehicle no wipgj8644 mcclhez6as71 make g wodel yamaha dt125 policy no ppoo21f066 name mra kg rupasinghe address 233191 no jaya mw kudamaduwa siddamulla 1gmar2023 15mar2024 period to cover of bfwo24477 engine no ismar 2fwo24477 chassis no docurnent the policy spected in conditons subjea and termns to")

Date: age 28 has an incorrect format.
Date: 233191 has an incorrect format.
Date: 2fwo24477 has an incorrect format.


In [ ]:
print(test5)

(100, 40.0, 0)


**Main_app Function 2**

In [ ]:
def main_app_functio2 (full_name:str,name_inti:str,address:str,birth_day:str,image_path):

  #final score
  name_score=''
  address_score=''
  age_score=""
  seal_validation=dict()
  #full_name="kaushika gihan Rupasinghe"
  #name_inti="K.G Rupasinghe"
  #address="No- 233/9/1, Jayamawatha rd,kudamaduwa, siddamulla,piliyandala,sri lanka"
  #birth_day="1995/16/oct"

  # Load the test image
  image = cv2.imread(image_path)

  # Preprocess the image
  if image is not None:

    easy_text=easy_ocr(image_path)
    keras_text=keras_ocr(image_path)

    print(easy_text)
    print(keras_text)

    easy_name_score,easy_address_score,easy_age_score=score_fun(full_name,name_inti,address,birth_day,easy_text)
    keras_name_score,keras_address_score,keras_age_score=score_fun(full_name,name_inti,address,birth_day,keras_text)

    print(easy_name_score)
    print(easy_address_score)
    print(easy_age_score)
    print(keras_name_score)
    print(keras_address_score)
    print(keras_age_score)



    #name final score
    if easy_name_score >= keras_name_score:
      name_score=easy_name_score

    else:
      name_score=keras_name_score



    #address final score

    if easy_address_score >= keras_address_score:
      address_score=easy_address_score

    else:
      address_score=keras_address_score



    #age final score

    if easy_age_score >= keras_age_score:
      age_score=easy_age_score

    else:
      age_score=keras_age_score

    ### seal verifications
    seal_validation=seal_verification(image_path)

  else:
    pass





  return int(name_score),int( address_score), int(age_score),seal_validation



In [ ]:
test5=main_app_functio2("kaushika gihan Rupasinghe","K.G Rupasinghe"," No- 233/9/1, Jayamawatha rd,kudamaduwa, siddamulla,piliyandala,sri lanka ","1995/16/oct","/content/drive/MyDrive/Work space/Data set/Healing Minds/fun2/predict data/IMG_20231020_235352.jpg")

2/2 [==============================] - 17s 9s/step
Wc ceylinco general insurance vp 'ceylinco house' 69, janadhipathi ltdeek outhe jert mawatta; coombo 1 certificate of insurance vehicie no wpgj8644 mcclh229046371 make & vodel yavaha dt125 policy %o pp00213f0008569 name m.k.g. rupasinghe address no : 233/9/1 jaya mw kudamaduwa siddamulla period of cover 16-mar-2023 to 16-mar-2024 engine no 3fw024477 chassis no 2fw024477 16mar subjedtto terrns and conditions specifed in tne policy docurrient
Cesinc ceylinco general insurance lidnud vid ceyinco house 69 janadhipath manatha coombol onele srat certificate of insurance vehicle no wpgje644 mcclh2z6616711 make r wodel yamaha dt125 policy no ppoo21f00666 name mre kg rupasinghe address no 233191 jaya mw kudamaduwa siddamulla igmarzo23 period to 5marz024 cover of 3fwo24477 engine no ismar 2fw024477 chassis no docurnent the policy spected in conditons subjec and to terms
Date: 233191 has an incorrect format.
Date: 5marz024 has an incorrect format

In [ ]:
print(test5)

(100, 60, 0, {'text': 'Valid seal verification is not included in this document', 'seal Class': ['No seal'], 'validation score': [98.70935082435608]})
